In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Final") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 23:08:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/03 23:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### 1. Data Ingestion
#### 2. Feature Selection
#### 3. Data Checks
#####     - check data type
#####     - check missing value
#####     - check unique values of each column
#####     - check statistics of dataset
#### 4. Data Saving/Loading
#### 5. Train Test Split
#### 6. Feature Engineering/Transformation
#### 7. Model Training and evaluation

### 1. Data Extraction

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [5]:
df = spark.read.text('2020_lar.txt')

In [6]:
df = spark.read.option("header", "true") \
    .option("delimiter", "|") \
    .csv('2020_lar.txt')

In [7]:
df

24/12/03 23:08:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
2020,549300RBX56T2MW5HO19,29620,MI,26065,26065003400,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,White,Joint,1,0,2,1,1,1,2,2,2,135000,97.0,3.875,1.273,2,460.0,NA,0.0,NULL,NA,360,NA,NA,2,2,2,2,145000,1,1,3,5,1,NA,65,30%-<36%,2,5,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,1,2,2,2,25-34,25-34,No,No,1,1,1,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,2253,28.72,79300,109.02,773,883,57
2020,549300RBX56T2MW5HO19,31084,CA,06037,06037108201,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,Asian,Joint,1,1,2,1,31,1,2,2,2,315000,36.82,2.124,-0.256,2,316.13,NA,316.13,NULL,316.13,180,NA,NA,2,2,2,2,855000,1,1,3,5,1,NA,135,20%-<30%,5,1,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,2,25,NULL,NULL,NULL,2,25,NULL,NULL,NULL,2,2,1,2,2,2,45-54,45-54,No,No,1,1,1,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,5846,62.37,77300,166.05,1378,1972,25
2020,549300RBX56T2MW5HO19,41100,UT,49053,49053270600,C,FHA:First Lien,Single Family (1-...,Not Hispanic or L...,White,Joint,1,9,2,2,1,1,1,2,2,135000,55.1,3.43,NA,3,NA,NA,NA,NA,NA,NA,NA,NA,2,2,2,2,255000,1,1,3,5,1,NA,NA,NA,9,9,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,1,2,2,2,65-74,65-74,Yes,Yes,1,1,6,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,7739,15.62,70700,102.72,1803,3813,19
2020,549300RBX56T2MW5HO19,19740,CO,08031,08031004004,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,White,Joint,4,0,2,1,32,1,2,2,2,405000,NA,NA,NA,3,NA,NA,NA,NA,NA,240,NA,NA,2,2,2,2,NA,1,1,3,5,1,NA,120,NA,9,9,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,1,2,2,2,35-44,35-44,No,No,1,1,6,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,1908,8.54,100000,145.9,802,760,52
2020,549300RBX56T2MW5HO19,17820,CO,08041,08041002501,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,White,Female,1,1,2,1,1,1,2,2,2,435000,80.0,2.5,-0.39,2,0.0,NA,0.0,NULL,NA,360,NA,NA,2,2,2,2,565000,1,1,3,5,1,NA,328,20%-<30%,1,2,2,NULL,NULL,NULL,NULL,2,NULL,

In [8]:
total_rows = df.count()

In [8]:
total_rows

25699043

In [9]:
df.columns

['activity_year',
 'lei',
 'derived_msa_md',
 'state_code',
 'county_code',
 'census_tract',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_ethnicity',
 'derived_race',
 'derived_sex',
 'action_taken',
 'purchaser_type',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'reverse_mortgage',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'interest_rate',
 'rate_spread',
 'hoepa_status',
 'total_loan_costs',
 'total_points_and_fees',
 'origination_charges',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'multifamily_affordable_units',
 'in

In [10]:
# sample subset for local machine
fraction = 10000/total_rows
df = df.sample(fraction=fraction, withReplacement=False)

### 2. Feature Selection

In [11]:
loan_details_df = df.select("activity_year", "lei", "loan_type","loan_purpose", "loan_amount", "interest_rate","loan_term", "action_taken")
borrower_details_df = df.select("income","applicant_age","applicant_sex", "applicant_credit_score_type", "co_applicant_age", "co_applicant_credit_score_type")
property_details_df = df.select("derived_msa_md", "state_code", "county_code", "property_value", "total_units", "occupancy_type")


In [12]:
columns = loan_details_df.columns + borrower_details_df.columns + property_details_df.columns

In [13]:
len(columns)

20

In [14]:
df = df.select(columns)

In [15]:
df

activity_year,lei,loan_type,loan_purpose,loan_amount,interest_rate,loan_term,action_taken,income,applicant_age,applicant_sex,applicant_credit_score_type,co_applicant_age,co_applicant_credit_score_type,derived_msa_md,state_code,county_code,property_value,total_units,occupancy_type
2020,549300RBX56T2MW5HO19,2,1,255000,3.75,360,1,75,45-54,1,1,<25,1,29620,MI,26045,265000,1,1
2020,549300RBX56T2MW5HO19,1,1,445000,2.75,360,1,140,35-44,1,5,35-44,5,17820,CO,08041,565000,1,1
2020,549300RBX56T2MW5HO19,1,1,385000,3.0,360,1,155,35-44,1,1,25-34,1,24540,CO,08123,495000,1,1
2020,549300RBX56T2MW5HO19,1,31,505000,3.375,360,1,186,35-44,1,5,35-44,1,14500,CO,08013,775000,1,1
2020,549300RBX56T2MW5HO19,1,32,285000,NA,360,4,66,55-64,1,9,55-64,9,19740,CO,08059,NA,1,1
2020,549300RBX56T2MW5HO19,1,31,465000,2.875,360,1,120,25-34,1,2,<25,2,42644,WA,53033,555000,1,1
2020,VMDPCBOQ43W3PZTYZL93,1,31,245000,NA,180,3,NA,35-44,3,9,9999,10,14454,MA,25025,NA,2,1
2020,549300CF8MP6S7MZV277,1,32,575000,3.875,360,1,157,45-54,1,3,9999,10,47894,DC,11001,1145000,1,1
2020,549300CF8MP6S7MZV277,1,31,775000,3.0,360,1,290,25-34,1,2,25-34,3,47894,DC,11001,1285000,1,1
2020,2549004OC91L41W9XR37,1,2,65000,4.0,300,1,138,35-44,1,8,9999,10,23420,CA,06019,585000,1,1


In [16]:
df = df.drop('activity_year')

### 3. Data Checks

#### Data Type

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType

from pyspark.sql.functions import col, when, count

In [10]:

coltype_map = {
    "lei": StringType(),
    "loan_type": IntegerType(),
    "loan_purpose": IntegerType(),
    "loan_amount": FloatType(),
    "interest_rate": FloatType(),
    "loan_term": IntegerType(),
    "action_taken": IntegerType(),
    "income": FloatType(),
    "applicant_age": StringType(),
    "applicant_sex": IntegerType(),
    "applicant_credit_score_type": IntegerType(),
    "co_applicant_age": StringType(),
    "co_applicant_credit_score_type": IntegerType(),
    "derived_msa_md": IntegerType(),
    "state_code": StringType(),
    "county_code": StringType(),
    "property_value": FloatType(),
    "total_units": IntegerType(),
    "occupancy_type": IntegerType(),
}



In [11]:
for col_name, data_type in coltype_map.items():
    df = df.withColumn(col_name, col(col_name).cast(data_type))

In [12]:
df.printSchema()

root
 |-- activity_year: string (nullable = true)
 |-- lei: string (nullable = true)
 |-- derived_msa_md: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- county_code: string (nullable = true)
 |-- census_tract: string (nullable = true)
 |-- conforming_loan_limit: string (nullable = true)
 |-- derived_loan_product_type: string (nullable = true)
 |-- derived_dwelling_category: string (nullable = true)
 |-- derived_ethnicity: string (nullable = true)
 |-- derived_race: string (nullable = true)
 |-- derived_sex: string (nullable = true)
 |-- action_taken: integer (nullable = true)
 |-- purchaser_type: string (nullable = true)
 |-- preapproval: string (nullable = true)
 |-- loan_type: integer (nullable = true)
 |-- loan_purpose: integer (nullable = true)
 |-- lien_status: string (nullable = true)
 |-- reverse_mortgage: string (nullable = true)
 |-- open_end_line_of_credit: string (nullable = true)
 |-- business_or_commercial_purpose: string (nullable = true)
 |-- lo

#### Missing values

In [13]:
from pyspark.sql.functions import col, isnan

# Missing count for each column
missing_info = df.select([
    count(when(col(c).isNull() | isnan(col(c)) |(col(c) == "NA"), c)).alias(c) for c in df.columns
])


In [14]:
missing_info

activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,0,0,191662,301904,363829,67537,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8860401,8058681,11061140,0,10157928,25032729,10058168,19789514,19261187,787724,24695256,23010226,0,0,0,0,6256159,0,0,0,0,67537,25104702,3805537,9141675,0,0,7909,24567966,25678698,25698254,25698741,2632,25253469,25691527,25698837,25698968,0,0,2422,24544261,25612871,25687495,25694850,923,25232356,25664488,25694290,25697228,0,0,0,0,0,0,0,0,2088192,14902695,0,0,0,23942490,25221397,25502047,25536392,0,25169448,25622071,25692311,0,0,0,0,0,0,0


In [15]:
# Median imputation for 'income'
from pyspark.ml.feature import Imputer

income_imputer = Imputer(inputCols=['income'], outputCols=['income'], strategy="median")
df = income_imputer.fit(df).transform(df)

In [16]:
# Mean imputation for `interest_rate`
interest_rate_imputer = Imputer(inputCols=["interest_rate"], outputCols=["interest_rate"], strategy="mean")

# Fit and transform
df = interest_rate_imputer.fit(df).transform(df)

In [17]:
# Mode imputation for 'loan_Term'
from pyspark.sql.functions import col, count, when

# Calculate mode for `loan_term`
mode_loan_term = (
    df.groupBy("loan_term")
    .count()
    .orderBy(col("count").desc())
    .first()[0]
)

# Replace missing values with the mode
df = df.withColumn( "loan_term", when(col("loan_term").isNull(), mode_loan_term).otherwise(col("loan_term")))


In [18]:
# Remove rows that has no 'property_value'
df = df.filter(col('property_value')!=0)

In [19]:
# Remove rows that has no 'state_code'
df = df.filter(col('state_code')!='NA')

In [20]:
# Remove rows that has no 'county_code'
df = df.filter(col('county_code')!='NA')

In [21]:
# Remove rows that has no 'total_units'
df = df.filter(col('total_units')!=0)

In [22]:
df.count()

19216487

In [23]:
df

activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
2020,549300RBX56T2MW5HO19,29620,MI,26065,26065003400,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,White,Joint,1,0,2,1,1,1,2,2,2,135000.0,97.0,3.875,1.273,2,460.0,NA,0.0,NULL,NA,360,NA,NA,2,2,2,2,145000.0,1,1,3,5,1,NA,65.0,30%-<36%,2,5,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,1,2,2,2,25-34,25-34,No,No,1,1,1,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,2253,28.72,79300,109.02,773,883,57
2020,549300RBX56T2MW5HO19,31084,CA,06037,06037108201,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,Asian,Joint,1,1,2,1,31,1,2,2,2,315000.0,36.82,2.124,-0.256,2,316.13,NA,316.13,NULL,316.13,180,NA,NA,2,2,2,2,855000.0,1,1,3,5,1,NA,135.0,20%-<30%,5,1,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,2,25,NULL,NULL,NULL,2,25,NULL,NULL,NULL,2,2,1,2,2,2,45-54,45-54,No,No,1,1,1,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,5846,62.37,77300,166.05,1378,1972,25
2020,549300RBX56T2MW5HO19,41100,UT,49053,49053270600,C,FHA:First Lien,Single Family (1-...,Not Hispanic or L...,White,Joint,1,9,2,2,1,1,1,2,2,135000.0,55.1,3.43,NA,3,NA,NA,NA,NA,NA,360,NA,NA,2,2,2,2,255000.0,1,1,3,5,1,NA,95.0,NA,9,9,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,1,2,2,2,65-74,65-74,Yes,Yes,1,1,6,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,7739,15.62,70700,102.72,1803,3813,19
2020,549300RBX56T2MW5HO19,17820,CO,08041,08041002501,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,White,Female,1,1,2,1,1,1,2,2,2,435000.0,80.0,2.5,-0.39,2,0.0,NA,0.0,NULL,NA,360,NA,NA,2,2,2,2,565000.0,1,1,3,5,1,NA,328.0,20%-<30%,1,2,2,NULL,NULL,NULL,NULL,2,NULL,NULL,NULL,NULL,2,2,5,NULL,NULL,NULL,NULL,5,NULL,NULL,NULL,NULL,2,2,2,2,2,2,55-64,55-64,No,No,1,1,1,NULL,NULL,NULL,NULL,10,NULL,NULL,NULL,3276,11.14,81700,146.97,1163,1472,45
2020,549300RBX56T2MW5HO19,11244,CA,06059,06059042334,C,Conventional:Firs...,Single Family (1-...,Not Hispanic or L...,Asian,Joint,1,1,2,1,31,1,2,2,2,355000.0,41.29,2.875,NA,2,120.0,NA,0.0,NULL,NA,180,NA,NA,2,2,2,2,855000.0,1,3,3

Target variable

In [24]:
df = df.withColumn('action_taken', when(col('action_taken')!=1, 0).otherwise(1) )

#### Check unique values of each column

In [25]:
df.select('action_taken').distinct()

action_taken
1
0


In [27]:
df.select('lei').distinct().count()

2957

In [28]:
df.select('applicant_age').distinct().count()

9

In [29]:
df.select('co_applicant_age').distinct().count()

9

In [30]:
df.select('state_code').distinct().count()

54

In [31]:
df.select('county_code').distinct().count()

3222

In [32]:
df.select('loan_type').distinct().count()

4

In [33]:
df.select('loan_purpose').distinct().count()

6

In [34]:
df.select('applicant_sex').distinct().count()

5

In [35]:
df.select('applicant_credit_score_type').distinct().count()

10

In [36]:
df.select('derived_msa_md').distinct().count()

413

In [37]:
df.select('total_units').distinct().count()

4

In [38]:
df.select('occupancy_type').distinct().count()

3

In [34]:
# from pyspark.sql.functions import col, isnan

# df.filter(col('action_taken').isNull()| isnan(col('action_taken')))

### Load data to parquet

In [35]:
df.write.parquet('./cleaned_dataset', mode='overwrite')

24/12/03 16:13:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/12/03 16:13:38 WARN MemoryManager: Total allocation exceeds 95.00% 